# Capstone Project - Find similar quarter in Sofia 
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## 1.Introduction <a name="introduction"></a>

### 1.1 Background

Sofia, as one of the most ancient cities in the Europe, is the largest city and the capital of Bulgaria.
Sofia hosts some 1.23 million residents within a territory of 492 km2, a concentration of 17.5% of the country population within the 200th percentile of the country territory. Sofia today becomes a new frontier for real estate investors,
For those who are looking for property in a lower price bracket, Sofia in Bulgaria is an excellent choice.
My friend, Wealth Management Expert present the Sofia real estate market to their clients in Paris.
He wants to show the client the Venue Data of Sofia.
His client actually resides in the eleventh arrondissement of Paris.
The eleventh arrondissement is a varied and engaging area. To the west lies the Place de la République, which is linked to the Place de la Bastille, in the east, by the sweeping, tree-lined Boulevard Richard-Lenoir, with its large markets and children's parks.

### 1.2 Business Problem

The business problem is that my friend will find the quater of Sofia which has the similar venue as the Paris 11e to show his client.

## 2. Data acquisition and cleaning<a name="data"></a>



### 2.1. Data Source :

#### Import necessary Libraries

In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


#### 2.1.1.Data Table : Sofia Quarter and Postalcode 

In [3]:
url='http://www.guide-bulgaria.com/SW/sofia-city/stolichna/sofia?t=postcodes'
html = requests.get(url).text
soup =bs(html,'lxml')
mylist= soup.find('div',{'class':'nine columns txt'}).find_all('li')
my_data=[]
mylist=[x.text.strip() for x in mylist]
my_data.append(mylist)
my_data=np.array(my_data).T.tolist()
df = pd.DataFrame(my_data,columns=["Data"])
new= df["Data"].str.split(' : ', n = 1, expand = True) 
df["Quarter"]= new[0]
df["Postalcode"]= new[1]
df.drop(columns =["Data"], inplace = True) 
df["Quarter"]= df["Quarter"].str.split(' ',n=1,expand=True).get(1)
df


,Quarter,Postalcode
0,Bakston,1618
1,Banishora,1233
2,Beli brezi,1680
3,Benkovski,1278
4,Borovo,1680
5,Botunets,1870
6,Boyana,1616
7,Chelopechene,1853
8,Darvenitsa,1756
9,Dianabad,1172


##### Use Nominatim to get geo coordinate for each quater (Test one quater works)

In [4]:
address = 'Lyulin 2, Sofia, Bulgaria'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)

latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

42.7256471 23.2501109


####  2.1.2.Get geocode for each quarter 

In [5]:
quartergeo=[]
latitude = []
longitude = []

for quarter in df["Quarter"]:
    try:
        inputAddress = '{},Sofia, Bulgaria'.format(quarter)
        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(inputAddress,timeout=20)
        quartergeo.append(quarter)
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except:
        latitude.append('NA')
        longitude.append('NA')
        print('Error, skipping address...')

df_geocodes = pd.DataFrame({'Quartergeo':quartergeo, 'Latitude':latitude,'Longitude':longitude})
df_geocodes.head()

Error, skipping address...
Error, skipping address...
Error, skipping address...
Error, skipping address...
Error, skipping address...
Error, skipping address...


,Quartergeo,Latitude,Longitude
0,Bakston,42.6655,23.27
1,Banishora,42.7104,23.3101
2,Beli brezi,42.6683,23.5991
3,Benkovski,42.7396,23.3461
4,Borovo,42.6694,23.2864


In [6]:
df_geocodes.head()

,Quartergeo,Latitude,Longitude
0,Bakston,42.6655,23.27
1,Banishora,42.7104,23.3101
2,Beli brezi,42.6683,23.5991
3,Benkovski,42.7396,23.3461
4,Borovo,42.6694,23.2864


In [7]:
left=df
right=df_geocodes
result = pd.merge(left, right, how='left', left_on='Quarter', right_on='Quartergeo')
result.drop(['Quartergeo'],axis=1,inplace=True)
result.dropna(inplace=True)
result['Latitude']= pd.to_numeric(result['Latitude'], errors='coerce')
result['Longitude']= pd.to_numeric(result['Longitude'], errors='coerce')
result = result[result['Longitude'].notna()]
result.reset_index(drop=True, inplace=True)
result

,Quarter,Postalcode,Latitude,Longitude
0,Bakston,1618,42.665486,23.269969
1,Banishora,1233,42.710366,23.310076
2,Beli brezi,1680,42.668338,23.599139
3,Benkovski,1278,42.739596,23.346105
4,Borovo,1680,42.669376,23.286375
5,Botunets,1870,42.739428,23.511576
6,Boyana,1616,42.646214,23.266749
7,Chelopechene,1853,42.732674,23.472598
8,Darvenitsa,1756,42.653300,23.361832
9,Dianabad,1172,42.662874,23.347948


In [8]:
result.shape

(77, 4)

In [10]:
address = 'Sofia, Bulgaria'
geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Sofia are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Sofia are 42.6978634, 23.3221789.


#### 2.1.3. Use Foursquare to get Sofia venue data

##### Define Foursquare Credentials and Version

In [11]:
CLIENT_ID = 'TKGWHMQGKH04YK4SERDJT0BAWNXEZAV5NGL5GACBCNDE4USU' # your Foursquare ID
CLIENT_SECRET = 'UEUZN2RRMSKQQOUM0AEYKH5VO022EPFDMMPAPGKO4ZLTG3ZH' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TKGWHMQGKH04YK4SERDJT0BAWNXEZAV5NGL5GACBCNDE4USU
CLIENT_SECRET:UEUZN2RRMSKQQOUM0AEYKH5VO022EPFDMMPAPGKO4ZLTG3ZH


In [12]:
result.loc[0, 'Quarter']

'Bakston'

In [13]:
quarter_latitude = result.loc[0, 'Latitude'] # latitude value
quarter_longitude = result.loc[0, 'Longitude'] # longitude value

quarter_name = result.loc[0, 'Quarter'] # quarter name

print('Latitude and longitude values of {} are {}, {}.'.format(quarter_name, 
                                                               quarter_latitude, 
                                                               quarter_longitude))

Latitude and longitude values of Bakston are 42.6654864, 23.269969.


##### First, let's create the GET request URL.

In [14]:
radius=500
limit=100
url= 'http://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID,
        CLIENT_SECRET,
        quarter_latitude,quarter_longitude,
        VERSION,
        radius,
        limit)

In [15]:
jresults = requests.get(url).json()

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
venues = jresults['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Аванти,Liquor Store,42.665564,23.272955
1,Млекарницата на Добрев 4,Cheese Shop,42.664005,23.266223
2,Ресторант Слънце,Restaurant,42.664546,23.266423
3,Сладкарница Сладки Идеи,Café,42.669091,23.269640
4,Dekora.Fashion,Clothing Store,42.663667,23.265996


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        jresults = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in jresults])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Quarter', 
                  'Quarter Latitude', 
                  'Quarter Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
Sofia_venues = getNearbyVenues(names=result['Quarter'],
                                   latitudes=result['Latitude'],
                                   longitudes=result['Longitude']
                                  )

Bakston
Banishora
Beli brezi
Benkovski
Borovo
Botunets
Boyana
Chelopechene
Darvenitsa
Dianabad
Dragalevtsi
Fakulteta
Filipovtsi
Geo Milev
Gorna Banya
Gorublyane
Gotse Delchev
Hadji Dimitar
Hipodruma
Hladilnika
Hristo Botev
Ivan Vazov
Izgrev
Iztok
Krasna Polyana
Krasno Selo
Kremikovtsi
Lagera
Levski
Lozenets
Lyulin 1
Lyulin 10
Lyulin 2
Lyulin 3
Lyulin 4
Lyulin 5
Lyulin 6
Lyulin 7
Lyulin 8
Lyulin 9
Malashevtsi
Malinova Dolina
Manastirski Livadi
Mladost 1
Mladost 2
Mladost 3
Mladost 4
Moderno predgradie
Motopista
Musagenitsa
Nadejda
Nadejda 3
Nadejda 6
Obelya
Obelya 2
Ovcha kupel
Ovcha kupel 2
Pavlovo
Poduyane
Poligona
Reduta
Republika
Seslavtsi
Simeonovo
Slatina
Stefan Karadja
Strelbishte
Studentski grad
Suhata Reka
Suhodol
Sveta Troitsa
Trebich
Yavorov
Zaharna fabrika
Zapaden Park
Zona B-18
Zona B-19


In [20]:
print(Sofia_venues.shape)
Sofia_venues.head()

(1430, 7)


,Quarter,Quarter Latitude,Quarter Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bakston,42.665486,23.269969,Аванти,42.665564,23.272955,Liquor Store
1,Bakston,42.665486,23.269969,Млекарницата на Добрев 4,42.664005,23.266223,Cheese Shop
2,Bakston,42.665486,23.269969,Ресторант Слънце,42.664546,23.266423,Restaurant
3,Bakston,42.665486,23.269969,Сладкарница Сладки Идеи,42.669091,23.269640,Café
4,Bakston,42.665486,23.269969,Dekora.Fashion,42.663667,23.265996,Clothing Store


#### 2.1.4. Use Foursquare to get Paris 11e  venue data

In [21]:
address = '75011 paris'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)

p_latitude = location.latitude
p_longitude = location.longitude
print(p_latitude, p_longitude)

48.8566969 2.3514616


In [22]:
radius=500
limit=100
url= 'http://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID,
        CLIENT_SECRET,
        p_latitude,p_longitude,
        VERSION,
        radius,
        limit)
jresults = requests.get(url).json()

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
venues = jresults['response']['groups'][0]['items']
    
paris11_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
paris11_venues =paris11_venues.loc[:, filtered_columns]

# filter the category for each row
paris11_venues['venue.categories'] = paris11_venues.apply(get_category_type, axis=1)

# clean columns
paris11_venues.columns = [col.split(".")[-1] for col in paris11_venues.columns]

paris11_venues.shape

(100, 4)

In [25]:
paris11_venues.head()

,name,categories,lat,lng
0,Place de l'Hôtel de Ville – Esplanade de la Li...,Plaza,48.856925,2.351412
1,Parc Rives de Seine,Park,48.855510,2.351419
2,L'Alsacien,Alsatian Restaurant,48.858275,2.350381
3,Berges de Seine – Rive droite,Pedestrian Plaza,48.855131,2.352289
4,Square de la Tour Saint-Jacques,Park,48.857882,2.348757


### 2.2 Data Cleaning 

In [26]:
paris11_venues['Quarter']='Paris11'
paris11_venues['Quarter Latitude']=p_latitude
paris11_venues['Quarter Longitude']=p_longitude
paris11_venues.head()

,name,categories,lat,lng,Quarter,Quarter Latitude,Quarter Longitude
0,Place de l'Hôtel de Ville – Esplanade de la Li...,Plaza,48.856925,2.351412,Paris11,48.856697,2.351462
1,Parc Rives de Seine,Park,48.855510,2.351419,Paris11,48.856697,2.351462
2,L'Alsacien,Alsatian Restaurant,48.858275,2.350381,Paris11,48.856697,2.351462
3,Berges de Seine – Rive droite,Pedestrian Plaza,48.855131,2.352289,Paris11,48.856697,2.351462
4,Square de la Tour Saint-Jacques,Park,48.857882,2.348757,Paris11,48.856697,2.351462


In summary of this data 100 venues returned by Foursquare. The Paris 11 arrondissment is popular quarter. If I want to find a similar quarter in Sofia, the quarter should have also 
**a lot of venues**.  

In [33]:

paris11_venues.columns = ['Venue','Venue Category','Venue Latitude','Venue Longitude','Quarter','Quarter Latitude','Quarter Longitude']

In [34]:
paris11_venues=paris11_venues[['Quarter','Quarter Latitude','Quarter Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category']]

In [110]:
paris11_venues.shape

(100, 7)

So only the quarter with **more than 30 venues** will be used. I named these quarters as popular quarters in sofia. 

In [126]:
sofia_count= Sofia_venues.groupby('Quarter').count()
sofia_count= sofia_count[sofia_count['Quarter Latitude']>30]
sofia_count.index

Index(['Bakston', 'Borovo', 'Geo Milev', 'Hladilnika', 'Ivan Vazov', 'Izgrev',
       'Iztok', 'Lozenets', 'Mladost 1', 'Mladost 4', 'Motopista',
       'Musagenitsa', 'Pavlovo', 'Poligona', 'Reduta', 'Stefan Karadja',
       'Strelbishte', 'Studentski grad', 'Yavorov', 'Zona B-18'],
      dtype='object', name='Quarter')

In [133]:
popquarter=sofia_count.index.tolist()
popquarter

['Bakston',
 'Borovo',
 'Geo Milev',
 'Hladilnika',
 'Ivan Vazov',
 'Izgrev',
 'Iztok',
 'Lozenets',
 'Mladost 1',
 'Mladost 4',
 'Motopista',
 'Musagenitsa',
 'Pavlovo',
 'Poligona',
 'Reduta',
 'Stefan Karadja',
 'Strelbishte',
 'Studentski grad',
 'Yavorov',
 'Zona B-18']

In [150]:
to_append=['Paris11','75011',p_latitude,p_longitude]
a_series = pd.Series(to_append, index = result.columns)
totalresult=result[result['Quarter'].isin(popquarter)].append(a_series, ignore_index=True)

I merge the venues datatable in popular sofia quarters  and the venue datatable in paris 11e.

In [134]:
Sofia_venues_pop=Sofia_venues[Sofia_venues['Quarter'].isin(popquarter)]

I remove one row with missing value.

In [135]:
total_venues=pd.concat([Sofia_venues_pop,paris11_venues])
total_venues.reset_index(drop=True, inplace=True)
total_venues.tail()

,Quarter,Quarter Latitude,Quarter Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
924,Paris11,49,2,Les Philosophes,49,2,French Restaurant
925,Paris11,49,2,Les Thermes de Lutèce,49,2,Salon / Barbershop
926,Paris11,49,2,Bières Cultes,49,2,Liquor Store
927,Paris11,49,2,Lafayette Anticipations,49,2,Art Gallery
928,Paris11,49,2,Place Georges Pompidou,49,2,Plaza


The data  needs to be represented in a numerically comparable way. To solve this, I  created a new column for each unique value in the “quarter” column. This is automatically done by the “get_dummies” function of Pandas. 

In [136]:
# one hot encoding
total_onehot = pd.get_dummies(total_venues[['Venue Category']], prefix="", prefix_sep="")

# add quarter column back to dataframe
total_onehot['Quarter'] = total_venues['Quarter'] 

# move quarter column to the first column
fixed_columns = [total_onehot.columns[-1]] + list(total_onehot.columns[:-1])
total_onehot = total_onehot[fixed_columns]

total_onehot.head()

,Quarter,Adult Boutique,Alsatian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Athletics & Sports,Auvergne Restaurant,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Beer Garden,Beer Store,Bike Trail,Bistro,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Bridal Shop,Buffet,Building,Bulgarian Restaurant,Burger Joint,Bus Station,Bus Stop,Cafeteria,Café,Canal,Candy Store,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Country Dance Club,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gymnastics Gym,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Lingerie Store,Liquor Store,Lottery Retailer,Lounge,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Nail Salon,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Perfume Shop,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Pool Hall,Portuguese Restaurant,Pub,Record Shop,Recreation Center,Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Steakhouse,Supermarket,Sushi Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Theater,Theme Park,Toy / Game Store,Track Stadium,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Wine Bar,Wine Shop,Winery,Yoga Studio
0,Bakston,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bakston,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Bakston,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bakston,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [137]:
total_grouped = total_onehot.groupby('Quarter').mean().reset_index()
total_grouped

,Quarter,Adult Boutique,Alsatian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Athletics & Sports,Auvergne Restaurant,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Beer Garden,Beer Store,Bike Trail,Bistro,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Bridal Shop,Buffet,Building,Bulgarian Restaurant,Burger Joint,Bus Station,Bus Stop,Cafeteria,Café,Canal,Candy Store,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Country Dance Club,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gymnastics Gym,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Lingerie Store,Liquor Store,Lottery Retailer,Lounge,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Nail Salon,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Perfume Shop,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Pool Hall,Portuguese Restaurant,Pub,Record Shop,Recreation Center,Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Steakhouse,Supermarket,Sushi Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Theater,Theme Park,Toy / Game Store,Track Stadium,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Wine Bar,Wine Shop,Winery,Yoga Studio
0,Bakston,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Borovo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Geo Milev,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hladilnika,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

## 3. Methodology <a name="methodology"></a>

I decided to use Kmeans algorithm.  
Kmeans algorithm is an iterative algorithm that tries to partition the dataset into Kpre-defined distinct non-overlapping subgroups (clusters) where each data point belongs to only one group. It tries to make the inter-cluster data points as similar as possible while also keeping the clusters as different as possible. It assigns data points to a cluster such that the sum of the squared distance between the data points and the cluster’s centroid (arithmetic mean of all the data points that belong to that cluster) is at the minimum. The less variation we have within clusters, the more homogeneous the data points are within the same cluster.

The way kmeans algorithm works is as follows:
- Specify number of clusters K.
- Initialize centroids by first shuffling the dataset and then randomly selecting K data points for the centroids without replacement.
- Keep iterating until there is no change to the centroids. i.e assignment of data points to clusters isn’t changing.
- Compute the sum of the squared distance between data points and all centroids.
- Assign each data point to the closest cluster (centroid).
- Compute the centroids for the clusters by taking the average of the all data points that belong to each cluster.

I will find Paris 11e will belong to which cluster. The remaining quarters in this cluster will be similar quarters in Sofia of Paris 11e.

## 4. Analysis <a name="analysis"></a>

Let's apply the data with the KMeans method. 

In [138]:
# set number of clusters
kclusters = 5

total_grouped_clustering = total_grouped.drop('Quarter', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(total_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 2, 4, 2, 1, 1, 2, 2, 3], dtype=int32)

In [139]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [152]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Quarter']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
quarters_venues_sorted = pd.DataFrame(columns=columns)
quarters_venues_sorted['Quarter'] = total_grouped['Quarter']

for ind in np.arange(total_grouped.shape[0]):
    quarters_venues_sorted.iloc[ind, 1:] = return_most_common_venues(total_grouped.iloc[ind, :], num_top_venues)

quarters_venues_sorted.head()

,Quarter,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bakston,Plaza,Eastern European Restaurant,Restaurant,Café,Farmers Market,Gym,Italian Restaurant,Cheese Shop,Chinese Restaurant,Snack Place
1,Borovo,Restaurant,BBQ Joint,Bakery,Pizza Place,Pharmacy,Café,Liquor Store,Italian Restaurant,Gym / Fitness Center,Yoga Studio
2,Geo Milev,Gym / Fitness Center,Eastern European Restaurant,Hotel,Dessert Shop,Sushi Restaurant,Bulgarian Restaurant,Bakery,Pizza Place,French Restaurant,Electronics Store
3,Hladilnika,Gym,Clothing Store,Dance Studio,Cosmetics Shop,Men's Store,Restaurant,Boxing Gym,Frozen Yogurt Shop,Nightclub,Coffee Shop
4,Ivan Vazov,Pizza Place,Italian Restaurant,Pharmacy,Bakery,Eastern European Restaurant,Yoga Studio,Nightclub,Café,Cheese Shop,Restaurant


In [153]:
# add clustering labels
quarters_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

totalresult.rename(columns={"Quarter":"Quarter"},inplace=True)
totalresult_merged = totalresult
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
totalresult_merged = totalresult_merged.join(quarters_venues_sorted.set_index('Quarter'), on='Quarter')

totalresult_merged.head(10) # check the last columns!

,Quarter,Postalcode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bakston,1618,43,23,3,Plaza,Eastern European Restaurant,Restaurant,Café,Farmers Market,Gym,Italian Restaurant,Cheese Shop,Chinese Restaurant,Snack Place
1,Borovo,1680,43,23,0,Restaurant,BBQ Joint,Bakery,Pizza Place,Pharmacy,Café,Liquor Store,Italian Restaurant,Gym / Fitness Center,Yoga Studio
2,Geo Milev,1574,43,23,2,Gym / Fitness Center,Eastern European Restaurant,Hotel,Dessert Shop,Sushi Restaurant,Bulgarian Restaurant,Bakery,Pizza Place,French Restaurant,Electronics Store
3,Hladilnika,1407,43,23,4,Gym,Clothing Store,Dance Studio,Cosmetics Shop,Men's Store,Restaurant,Boxing Gym,Frozen Yogurt Shop,Nightclub,Coffee Shop
4,Ivan Vazov,1408,43,23,2,Pizza Place,Italian Restaurant,Pharmacy,Bakery,Eastern European Restaurant,Yoga Studio,Nightclub,Café,Cheese Shop,Restaurant
5,Izgrev,1113,43,23,1,Restaurant,Eastern European Restaurant,Health & Beauty Service,Italian Restaurant,Chinese Restaurant,Park,Nail Salon,Juice Bar,Garden,Bulgarian Restaurant
6,Iztok,1113,43,23,1,Bulgarian Restaurant,Eastern European Restaurant,Gym,Italian Restaurant,Restaurant,Health & Beauty Service,Fast Food Restaurant,Nail Salon,Dessert Shop,Dance Studio
7,Lozenets,1164 / 1421,43,23,2,Grocery Store,Café,Restaurant,Hotel,Bulgarian Restaurant,Pizza Place,Sushi Restaurant,Beer Garden,Soccer Field,Spa
8,Mladost 1,1750 / 1784,43,23,2,Restaurant,Pizza Place,Supermarket,Gastropub,Café,Bus Stop,Bus Station,Fast Food Restaurant,Bulgarian Restaurant,Farmers Market
9,Mladost 4,1715,43,23,3,Italian Restaurant,Restaurant,Pizza Place,Café,Supermarket,Bistro,Ethiopian Restaurant,Buffet,Pharmacy,Shipping Store


In [154]:
totalresult_merged.dropna(inplace=True)
totalresult_merged.reset_index(drop=True, inplace=True)

The Paris 11 belongs to Cluster Label N°2.

In [155]:
totalresult_merged.loc[totalresult_merged['Quarter'] == 'Paris11']

,Quarter,Postalcode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Paris11,75011,49,2,2,French Restaurant,Plaza,Ice Cream Shop,Art Gallery,Hotel,Clothing Store,Cocktail Bar,Tea Room,Bar,Park


There are **6** quarters in Sofia in the cluster N°2.

In [156]:
totalresult_merged.loc[totalresult_merged['Cluster Labels'] == 2]

,Quarter,Postalcode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Geo Milev,1574,43,23,2,Gym / Fitness Center,Eastern European Restaurant,Hotel,Dessert Shop,Sushi Restaurant,Bulgarian Restaurant,Bakery,Pizza Place,French Restaurant,Electronics Store
4,Ivan Vazov,1408,43,23,2,Pizza Place,Italian Restaurant,Pharmacy,Bakery,Eastern European Restaurant,Yoga Studio,Nightclub,Café,Cheese Shop,Restaurant
7,Lozenets,1164 / 1421,43,23,2,Grocery Store,Café,Restaurant,Hotel,Bulgarian Restaurant,Pizza Place,Sushi Restaurant,Beer Garden,Soccer Field,Spa
8,Mladost 1,1750 / 1784,43,23,2,Restaurant,Pizza Place,Supermarket,Gastropub,Café,Bus Stop,Bus Station,Fast Food Restaurant,Bulgarian Restaurant,Farmers Market
11,Musagenitsa,1797,43,23,2,Café,Restaurant,Pharmacy,Athletics & Sports,Electronics Store,Eastern European Restaurant,BBQ Joint,Paper / Office Supplies Store,Dessert Shop,Supermarket
17,Studentski grad,1700,43,23,2,Pizza Place,Supermarket,Nightclub,Bar,Cocktail Bar,Electronics Store,Gym,Cosmetics Shop,Fast Food Restaurant,Dessert Shop
20,Paris11,75011,49,2,2,French Restaurant,Plaza,Ice Cream Shop,Art Gallery,Hotel,Clothing Store,Cocktail Bar,Tea Room,Bar,Park


In [157]:
result_merged= totalresult_merged[totalresult_merged.Quarter != 'Paris11']

In [158]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11) 

#set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(result_merged['Latitude'], result_merged['Longitude'], result_merged['Quarter'], result_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

All the quarters similar to Paris 11e are the **blue points** in the Sofia map.

## 5. Results and Discussion <a name="results"></a>

In this study, I used the venues information to compare one quarter in Paris with quarters in Sofia. Result of this analysis, there are 6 quarters which are the most similar to Paris 11e. My friend  can show his clients firstly the appartments in these 6 quarters.

## 6. Conclusion <a name="conclusion"></a>

This analysis can use to find potential deal and present the location which is interested in the investor.